## 셀레니움으로 스크래핑

In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd 
from collections import OrderedDict
import time
import requests
from scrapy.http import TextResponse
pd.set_option('display.max_colwidth', None)

In [2]:
url = 'https://www.melon.com/chart/hot100/index.htm'
chrome_path = 'C:/Users/rnlay/.wdm/drivers/chromedriver/win64/118.0.5993.89/chromedriver.exe'

options = webdriver.ChromeOptions()
options.add_argument(f'--executable-path={chrome_path}')

driver = webdriver.Chrome(options=options)

driver.implicitly_wait(10)
driver.get(url)

In [3]:
# 해당 페이지 사용. bs를 활용
html = driver.page_source
soup = bs(html, 'html.parser')
song_soup = soup.select('tbody > tr')

In [4]:
song_lst = []

for song in song_soup:
    song_title = song.find('div', class_= 'ellipsis rank01').get_text()
    song_no = song.find('div', class_='ellipsis rank01').find('a')['href'].split("'")[2]
    song_artist = song.find('span', class_= 'checkEllipsis').get_text() # ellipsis rank02로 가져오면 아티스트가 2번출력됨
    song_album = song.find('div', class_= 'ellipsis rank03').get_text()
    song_likes = song.find('button', class_ = 'button_etc like').get_text()
    song_img = song.find('a', class_='image_typeAll').find('img')['src']

    song_lst.append([song_no, song_title.replace('\n', ''), song_artist.replace('\n', ''), song_album.replace('\n', ''), song_likes.replace('\n', '').replace('좋아요총건수', ''), song_img])

In [5]:
driver.quit()

df = pd.DataFrame(song_lst, columns = ['no', 'title', 'artist', 'album', 'likes', 'img'])

df['no'] = df['no'].str.extract(r'(\d+)')
df['순위'] = range(1, len(df) + 1)

In [6]:
df = df.rename(columns = {'no': '곡ID', 'title': '곡명', 'artist': '가수', 'album': '앨범', 'likes': '좋아요', 'img': '앨범자켓'})
df = df[['순위', '곡ID','곡명', '가수', '앨범', '좋아요', '앨범자켓']]
df.head(10)

,순위,곡ID,곡명,가수,앨범,좋아요,앨범자켓
0,1,37194943,밤양갱,비비 (BIBI),밤양갱,"76,975",https://cdnimg.melon.co.kr/cm2/album/images/11...
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’,"68,844",https://cdnimg.melon.co.kr/cm2/album/images/11...
2,3,37145732,Love wins all,아이유,The Winning,"135,212",https://cdnimg.melon.co.kr/cm2/album/images/11...
3,4,37225604,EASY,LE SSERAFIM (르세라핌),EASY,"34,954",https://cdnimg.melon.co.kr/cm2/album/images/11...
4,5,37003911,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,"126,446",https://cdnimg.melon.co.kr/cm2/album/images/11...
5,6,37023625,비의 랩소디,임재현,비의 랩소디,"54,242",https://cdnimg.melon.co.kr/cm2/album/images/11...
6,7,37222942,홀씨,아이유,The Winning,"32,359",https://cdnimg.melon.co.kr/cm2/album/images/11...
7,8,37248283,WAY 4 LUV,PLAVE,ASTERUM : 134-1,"27,121",https://cdnimg.melon.co.kr/cm2/album/images/11...
8,9,37098744,Love 119,RIIZE,Love 119,"82,090",https://cdnimg.melon.co.kr/cm2/album/images/11...
9,10,37248282,Watch Me Woo!,PLAVE,ASTERUM : 134-1,"21,085",https://cdnimg.melon.co.kr/cm2/album/images/11...


In [8]:
df['앨범자켓'].sample(10)

93    https://cdnimg.melon.co.kr/cm2/album/images/113/85/105/11385105_20231214151154_500.jpg/melon/resize/120/quality/80/optimize
61    https://cdnimg.melon.co.kr/cm2/album/images/113/91/082/11391082_20231228140237_500.jpg/melon/resize/120/quality/80/optimize
37    https://cdnimg.melon.co.kr/cm2/album/images/114/23/457/11423457_20240226121537_500.jpg/melon/resize/120/quality/80/optimize
87    https://cdnimg.melon.co.kr/cm2/album/images/113/72/658/11372658_20231124153648_500.jpg/melon/resize/120/quality/80/optimize
52    https://cdnimg.melon.co.kr/cm2/album/images/114/24/984/11424984_20240222142756_500.jpg/melon/resize/120/quality/80/optimize
48    https://cdnimg.melon.co.kr/cm2/album/images/114/02/316/11402316_20240118165756_500.jpg/melon/resize/120/quality/80/optimize
38    https://cdnimg.melon.co.kr/cm2/album/images/114/11/500/11411500_20240202172143_500.jpg/melon/resize/120/quality/80/optimize
58    https://cdnimg.melon.co.kr/cm2/album/images/113/73/371/11373371_20231128100602_500.j

In [9]:
# 이미지 가져오기
song_img = song.find('a', class_='image_typeAll').find('img')['src']
song_img

'https://cdnimg.melon.co.kr/cm2/album/images/113/97/942/11397942_20240111152314_500.jpg/melon/resize/120/quality/80/optimize'

In [10]:
from IPython.display import display, Image

def display_image(image_url):
    display(Image(url=image_url))

In [11]:
image_url = song_img

display_image(image_url)

## 가사 추출

In [24]:
df_ID = df[['순위', '곡ID', '곡명', '가수']]
df_ID

,순위,곡ID,곡명,가수
0,1,37194943,밤양갱,비비 (BIBI)
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스)
2,3,37145732,Love wins all,아이유
3,4,37225604,EASY,LE SSERAFIM (르세라핌)
4,5,37003911,To. X,태연 (TAEYEON)
...,...,...,...,...
95,96,37070241,그 겨울엔 네가 있어,"허용별 (허각, 신용재, 임한별)"
96,97,37173722,I GOT YOU,TWICE (트와이스)
97,98,37265919,사랑은 하니까 (Prod. 최유리) (inst.),홍이삭
98,99,37257772,Easy,숀 (SHAUN)


In [25]:
def get_lyrics(song_id):
    headers = {"user-agent": UserAgent().chrome}
    url = f"https://www.melon.com/song/detail.htm?songId={song_id}"
    res = requests.get(url, headers=headers)
    
    if res.status_code == 200:
        soup = bs(res.text, 'html.parser') # bs: BeautifulSoup
        lyrics_tag = soup.find('div', class_='lyric')
        
        if lyrics_tag:
            lyrics = lyrics_tag.get_text(separator=' ', strip=True)
            return lyrics

    return None


df2 = pd.DataFrame(df_ID)
df2['가사'] = df2['곡ID'].apply(get_lyrics) # 해당 ID의 가사 추가
df2['가사'] = df2['가사'].str.replace('\n', ' ')
df2

,순위,곡ID,곡명,가수,가사
0,1,37194943,밤양갱,비비 (BIBI),떠나는 길에 니가 내게 말했지 ‘너는 바라는 게 너무나 많아 잠깐이라도 널 안 바라보면 머리에 불이 나버린다니까’ 나는 흐르려는 눈물을 참고 하려던 얘길 어렵게 누르고 ‘그래 미안해’라는 한 마디로 너랑 나눈 날들 마무리했었지 달디달고 달디달고 달디단 밤양갱 밤양갱 내가 먹고 싶었던 건 달디단 밤양갱 밤양갱이야 떠나는 길에 니가 내게 말했지 ‘너는 바라는 게 너무나 많아’ 아냐 내가 늘 바란 건 하나야 한 개뿐이야 달디단 밤양갱 달디달고 달디달고 달디단 밤양갱 밤양갱 내가 먹고 싶었던 건 달디단 밤양갱 밤양갱이야 상다리가 부러지고 둘이서 먹다 하나가 쓰러져버려도 나라는 사람을 몰랐던 넌 떠나가다가 돌아서서 말했지 ‘너는 바라는 게 너무나 많아’ 아냐 내가 늘 바란 건 하나야 한 개뿐이야 달디단 밤양갱
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),"Ay ay ay ay ay 거울 속에 내 표정 봐 봐 느낌 So good 기다려온 D-day 연습했던 손든 인사도 그대로 하면 돼 Hairstyle check하고 한 번 Turn around 발걸음은 매일 걷던 그 길로 계획은 완벽 빨리 말 걸어보고 싶어, Hey Woo 문 앞에서 셋을 세어본다, Yeh (셋, 둘, 하나) 첫 만남은 너무 어려워 계획대로 되는 게 없어서 첫 만남은 너무 어려워 내 이름은 말야 Hey, 안녕, 첫 마디를 건넬 때 주변 소린 Canceled 네 말소리는 Playlist Yeh, 질문은 나의 용기, 알려줘 너의 “이름이 뭐야?” 너와 내 거리는 세 걸음 남았어, Yeh (셋, 둘, 하나) 첫 만남은 너무 어려워 계획대로 되는 게 없어서 첫 만남은 너무 어려워 내 이름은 말야 이 순간, Feels so wonderful 조금은 뚝딱거려도 어색한 인사까지도 너와 나의 첫 만남 우리의 사이 Beautiful 내일도 내일모레도 기억해, 영원히 반짝일 순간 Wait wait! Na na na- 이렇게 만나서 반가워 내일 또 봐 안녕"
2,3,37145732,Love wins all,아이유,"Dearest, Darling, My universe 날 데려가 줄래? 나의 이 가난한 상상력으론 떠올릴 수 없는 곳으로 저기 멀리 from Earth to Mars 꼭 같이 가줄래? 그곳이 어디든, 오랜 외로움 그 반대말을 찾아서 어떤 실수로 이토록 우리는 함께일까 세상에게서 도망쳐 Run on 나와 저 끝까지 가줘 My lover 나쁜 결말일까 길 잃은 우리 둘 um 부서지도록 나를 꼭 안아 더 사랑히 내게 입 맞춰 Lover Love is all Love is all Love Love Love Love 결국, 그럼에도, 어째서 우리는 서로일까 세상에게서 도망쳐 Run on 나와 저 끝까지 가줘 My lover 나쁜 결말일까 길 잃은 우리 둘 um 찬찬히 너를 두 눈에 담아 한 번 더 편안히 웃어주렴 유영하듯 떠오른 그날 그 밤처럼, 나와 함께 겁 없이 저물어줄래? 산산히 나를 더 망쳐 Ruiner 너와 슬퍼지고 싶어 My lover 필연에게서 도망쳐 Run on 나와 저 끝까지 가줘 My lover 일부러 나란히 길 잃은 우리 두 사람 부서지도록 나를 꼭 안아 더 사랑히 내게 입 맞춰 Lover Our Love wins all Love wins all Love Love Love Love"
3,4,37225604,EASY,LE SSERAFIM (르세라핌),"다친대도 길을 걸어 kiss me 쉽지 않음 내가 쉽게 easy Stage 위엔 불이 튀어, 내 body Pull up and I rip it up like ballet Damn, I really make it look easy Yuh know that I make it look easy Clap your hands, 너도 나와 같다면 clap your hands now 의구심 따윈 그냥 치워, 그냥 말해 “I’m the real one” 조명 꺼진 뒤의 난 wander in the night Don’t know what is right, don’t know ‘bout my rights 시기심, 의심, 불신, 이젠 friends of me, yuh 세상에게 난 반쪽짜리 seraphim, yuh Uhm I’ve been tryin’ so long to show you show you show you I’m the one that you need 다친대도 길을 걸어 kiss me 쉽지 않음 내가 쉽게 easy Stage 위엔 불이 튀어, 내 body Pull up and I rip it up like ballet Damn, I really make it look easy Yuh know that I make it look easy Yah know that I make it look easy 수면 위의 백조가 돼 whippin’ 때론 풀려, 나의 다리 but I keep it Get, set, go, come and see me, I’m the FEARLESS 나의 발걸음은 매 순간 history, 이건 my way 영웅처럼 걸어 even if I am not flawless 편하게만 왔다고? 날 몰라봤다면 you have to know Uhm I’ve been tryin’ so long to show you show you show you I’m the one that you need 다친대도 길을 걸어 kiss me 쉽지 않음 내가 쉽게 easy Stage 위엔 불이 튀어, 내 body Pull up and I rip it up like ballet Damn, I really make it look easy Yuh know that I make it look easy"
4,5,37003911,To. X,태연 (TAEYEON),"처음 본 널 기억해 We skipped the small talk 바로 다음 단계였지 뭐 단점이라곤 없는 게 단점이라던 그 허세도 마냥 좋았어 하지만 내 일기가 재미없어진 이유 내가 없어진 나의 매일들은 허전해 좀 이상해 왜 둘 사이에 너만 너만 보이는 걸까 난 까다롭고 힘든 아이라 그런 피곤한 생각만 한대 오늘 나눈 문자 속에 새로 산 티셔츠 그 얘기뿐이야 이제야 난 알 것 같아 Gonna block you 불을 꺼 To. X 그 좋아했던 립스틱 싫단 말에 버린 널 좋아했던 만큼 다 맞추려 했어 ‘나 아님 누가 그런 세상 안아주겠어’ 그 말이 그땐 그리 달콤했던 거야 Oh no 새벽의 긴 통화도 이젠 피곤해졌어 Every day, every night 나로 채우고 싶어 좀 이상해 왜 둘 사이에 너만 너만 보이는 걸까 난 까다롭고 힘든 아이라 그런 피곤한 생각만 한대 오늘 나눈 문자 속에 새로 산 티셔츠 그 얘기뿐이야 이제야 난 알 것 같아 Gonna block you 불을 꺼 To. X Gonna block you 불을 꺼 To. X Say Bye Gonna block you 불을 꺼 To. X 깜빡거리는 흔들거리는 Light Gonna block you 불을 꺼 To. X 좀 이상해 왜 둘 사이에 너만 너만 보이는 걸까 (너만 보일까) 난 까다롭고 힘든 아이라 그런 피곤한 생각만 한대 오늘 나눈 문자 속에 (이제 불을 꺼) 새로 산 티셔츠 그 얘기뿐이야 이제야 난 알 것 같아 Gonna block you 불을 꺼 To. X Gonna block you 불을 꺼 To. X"
...,...,...,...,...,...
95,96,37070241,그 겨울엔 네가 있어,"허용별 (허각, 신용재, 임한별)",찬 바람에 떠밀리듯 걸음을 옮기다 움츠렸던 기억들을 마주하는 밤 삐뚤어진 목도리를 고쳐주던 네가 바보처럼 또 생각나 시려오는 코끝에 훌쩍이는 맘에 너의 얼굴이 스치듯 떠올라 고마워 생각나 줘서 내 겨울이 돼줘서 꼭 한 번쯤은 첫눈처럼 내려줘서 반가운 너를 따라 그 시절로 되감아 보면 그곳엔 네가 있어 여전히 웃고 있어 불빛 가득 거리에 종이 울려오면 함께 부르던 12월의 노래 고마워 생각나 줘서 내 겨울이 돼줘서 꼭 한 번쯤은 첫눈처럼 내려줘서 반가운 너를 따라 그 시절로 되감아 보면 그곳엔 네가